# Initial data cleaning (Part 2)

In this notebook we do some data cleaning for a small portion of the POGOH dataset, this will give some ideas on how to proceed for dealing with the data at a larger scale.

__NOTE:__ In this dataset there were some NaN observations in End Station Id and End Station Name, and because of this the ID column is read as a float instead of integer. We handle this issue in this notebook.

In [1]:
import pandas as pd
import string
import sys
sys.path.append('/home/manuel/Documents/AI/pogoh-ai-engineering')
import shared_utils.pogoh_cleaning as pc

# Define the file path
file_path = "/home/manuel/Documents/AI/pogoh-ai-engineering/data/raw/april-2025.xlsx"
# Load the Excel file into a DataFrame
pogoh_df = pd.read_excel(file_path)
pogoh_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47523 entries, 0 to 47522
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Closed Status       47523 non-null  object        
 1   Duration            47523 non-null  int64         
 2   Start Station Id    47523 non-null  int64         
 3   Start Date          47523 non-null  datetime64[ns]
 4   Start Station Name  47523 non-null  object        
 5   End Date            47523 non-null  datetime64[ns]
 6   End Station Id      47497 non-null  float64       
 7   End Station Name    47497 non-null  object        
 8   Rider Type          47523 non-null  object        
dtypes: datetime64[ns](2), float64(1), int64(2), object(4)
memory usage: 3.3+ MB


In [2]:
pogoh_df_cleaned = pc.clean_column_names(pogoh_df)
pogoh_df_cleaned.columns

Index(['closed_status', 'duration', 'start_station_id', 'start_date',
       'start_station_name', 'end_date', 'end_station_id', 'end_station_name',
       'rider_type'],
      dtype='object')

First we'll create a function *_check_duration_mismatch()* that checks if the information between *duration*, *start date* and *end date* matches. The duration of the trip is measured seconds, so we are matching this with the difference betwen the end and start times.

In [3]:
pogoh_df_cleaned['duration'].describe()

count     47523.000000
mean        790.384277
std        2694.794272
min           0.000000
25%         222.000000
50%         381.000000
75%         821.000000
max      200129.000000
Name: duration, dtype: float64

In [4]:
pogoh_df_cleaned[['duration','start_date','end_date']].head()

,duration,start_date,end_date
0,412,2025-04-30 23:58:19,2025-05-01 00:05:11
1,179,2025-04-30 23:58:06,2025-05-01 00:01:05
2,1060,2025-04-30 23:48:29,2025-05-01 00:06:09
3,1173,2025-04-30 23:46:30,2025-05-01 00:06:03
4,394,2025-04-30 23:45:03,2025-04-30 23:51:37


In [5]:
pc._check_duration_mismatch(pogoh_df_cleaned, tolerance_sec=0).sum()

np.int64(0)

In [6]:
# Sample data with one mismatch in row 2
data = {
    "start_date": [
        "2025-06-17 08:00:00",
        "2025-06-17 09:15:00",
        "2025-06-17 10:00:00",
        "2025-06-17 11:45:00"
    ],
    "end_date": [
        "2025-06-17 08:30:00",  # 1800 seconds
        "2025-06-17 09:45:00",  # 1800 seconds
        "2025-06-17 10:20:00",  # 1200 seconds, but duration says 900 (mismatch)
        "2025-06-17 12:30:00"   # 2700 seconds
    ],
    "duration": [
        1800,  # match
        1800,  # match
        900,   # mismatch!
        2700   # match
    ]
}

df_test = pd.DataFrame(data)
pc._check_duration_mismatch(df_test).sum()


np.int64(1)

The next function, *_check_negative_or_zero_duration*,checks if there were any trips with negative or 0 duration.

In [7]:
pc._check_negative_or_zero_duration(pogoh_df_cleaned).sum()

np.int64(1)

The *_check_unrealistic_duration* function looks for trips longer than a specified duration. The default is one day.

In [8]:
pc._check_unrealistic_duration(pogoh_df_cleaned, max_seconds=86400).sum()

np.int64(8)

The *_check_missing_timestamps* function checks for rows where the timestamps for start or end of the trip are missing.

In [9]:
pc._check_missing_timestamps(pogoh_df_cleaned).sum()

np.int64(0)

The *_check_start_after_end* function checks for rows where the start timestamp is after the end timestamp.

In [10]:
pc._check_start_after_end(pogoh_df_cleaned).sum()

np.int64(0)

The *_check_datetime_out_of_range* function checks for rows where the start/end timestamps are outside of a defined range.

In [11]:
pc._check_datetime_out_of_range(pogoh_df_cleaned, min_year=2020, max_year=2025, min_month=1, max_month=12).sum()

np.int64(0)

The *validate_trip_data* function creates a dictionary with information about the number of rows that present any issues with the predefined checks.

In [12]:
summary = pc.validate_trip_data(pogoh_df_cleaned)
pc.summarize_trip_validation(summary)

Trip Data Validation Summary:
-----------------------------------
[FAIL]  Negative or zero duration: 1 issue(s) found.
[FAIL]  Unrealistic duration: 8 issue(s) found.
[OK]    Missing timestamps: no issues found.
[OK]    Start after end: no issues found.
[OK]    Datetime out of range: no issues found.
[OK]    Duration mismatch: no issues found.
-----------------------------------


The *get_trip_data_issues* function Returns a DataFrame of rows with validation issues according to the above defined functions. It can return the flags for troublesome rows only or include the full original data for such observations.

In [13]:
pc.get_trip_data_issues(pogoh_df_cleaned, include_full_data=True)

,closed_status,duration,start_station_id,start_date,start_station_name,end_date,end_station_id,end_station_name,rider_type,original_index,invalid_negative_or_zero_duration,invalid_unrealistic_duration,invalid_missing_timestamps,invalid_start_after_end,invalid_datetime_out_of_range,invalid_duration_mismatch
0,TERMINATED,175669,37,2025-04-22 14:54:11,S Negley Ave & Centre Ave,2025-04-24 15:42:00,NaN,NaN,MEMBER,16206,False,True,False,False,False,False
1,NORMAL,99963,31,2025-04-18 08:04:19,First Ave & B St,2025-04-19 11:50:22,22.0,Forbes Ave & Market Square,MEMBER,24137,False,True,False,False,False,False
2,NORMAL,97753,26,2025-04-17 23:20:42,S 27th St & Sidney St. (Southside Works),2025-04-19 02:29:55,42.0,Second Ave & Tecumseh St,CASUAL,24295,False,True,False,False,False,False
3,NORMAL,148447,24,2025-04-15 16:00:52,Penn Ave & 33rd St,2025-04-17 09:14:59,24.0,Penn Ave & 33rd St,MEMBER,27975,False,True,False,False,False,False
4,NORMAL,200129,24,2025-04-14 12:58:23,Penn Ave & 33rd St,2025-04-16 20:33:52,24.0,Penn Ave & 33rd St,MEMBER,30226,False,True,False,False,False,False
5,TERMINATED,0,33,2025-04-13 20:57:17,Schenley Dr & Schenley Dr Ext,2025-04-13 20:57:17,33.0,Schenley Dr & Schenley Dr Ext,MEMBER,30912,True,False,False,False,False,False
6,NORMAL,169154,23,2025-04-09 13:42:12,North Shore Trail & Fort Duquesne Bridge,2025-04-11 12:41:26,24.0,Penn Ave & 33rd St,CASUAL,36682,False,True,False,False,False,False
7,NORMAL,92908,13,2025-04-06 14:32:54,S Bouquet Ave & Sennott St,2025-04-07 16:21:22,24.0,Penn Ave & 33rd St,CASUAL,40312,False,True,False,False,False,False
8,NORMAL,91478,13,2025-04-06 14:32:47,S Bouquet Ave & Sennott St,2025-04-07 15:57:25,36.0,Penn Ave & S Pacific Ave,CASUAL,40313,False,True,False,False,False,False


In [14]:
pc.get_trip_data_issues(pogoh_df_cleaned, include_full_data=False)

,invalid_negative_or_zero_duration,invalid_unrealistic_duration,invalid_missing_timestamps,invalid_start_after_end,invalid_datetime_out_of_range,invalid_duration_mismatch,original_index
0,False,True,False,False,False,False,16206
1,False,True,False,False,False,False,24137
2,False,True,False,False,False,False,24295
3,False,True,False,False,False,False,27975
4,False,True,False,False,False,False,30226
5,True,False,False,False,False,False,30912
6,False,True,False,False,False,False,36682
7,False,True,False,False,False,False,40312
8,False,True,False,False,False,False,40313


Here I illustrate the use of the functions with a synthetic dataset.

In [15]:
df_fail = pd.DataFrame({
    "start_date": [
        "2025-06-17 08:00:00",  # OK
        "2025-06-17 10:00:00",  # OK
        None,                   # missing start timestamp
        "2025-06-17 12:00:00",  # start after end
        "2015-01-01 08:00:00",  # year/month out of range
        "2025-06-17 08:00:00"   # OK
    ],
    "end_date": [
        "2025-06-17 08:05:00",  # OK
        "2025-06-17 10:01:00",  # duration mismatch
        "2025-06-17 11:00:00",  # OK
        "2025-06-17 11:00:00",  # end before start
        "2015-01-01 09:00:00",  # year/month out of range
        "2025-06-18 08:01:00"   # > 24 hours
    ],
    "duration": [
        0,        # negative or zero
        30,       # duration mismatch
        3600,     # OK
        300,      # OK
        3600,     # OK but dates out of range
        86500     # unrealistic (> 86400)
    ]
})


In [16]:
summary_fail = pc.validate_trip_data(df_fail)
pc.summarize_trip_validation(summary_fail)

Trip Data Validation Summary:
-----------------------------------
[FAIL]  Negative or zero duration: 1 issue(s) found.
[FAIL]  Unrealistic duration: 1 issue(s) found.
[FAIL]  Missing timestamps: 1 issue(s) found.
[FAIL]  Start after end: 1 issue(s) found.
[FAIL]  Datetime out of range: 2 issue(s) found.
[FAIL]  Duration mismatch: 4 issue(s) found.
-----------------------------------


In [17]:
pc.get_trip_data_issues(df_fail)


,start_date,end_date,duration,original_index,invalid_negative_or_zero_duration,invalid_unrealistic_duration,invalid_missing_timestamps,invalid_start_after_end,invalid_datetime_out_of_range,invalid_duration_mismatch
0,2025-06-17 08:00:00,2025-06-17 08:05:00,0,0,True,False,False,False,False,True
1,2025-06-17 10:00:00,2025-06-17 10:01:00,30,1,False,False,False,False,False,True
2,None,2025-06-17 11:00:00,3600,2,False,False,True,False,True,False
3,2025-06-17 12:00:00,2025-06-17 11:00:00,300,3,False,False,False,True,False,True
4,2015-01-01 08:00:00,2015-01-01 09:00:00,3600,4,False,False,False,False,True,False
5,2025-06-17 08:00:00,2025-06-18 08:01:00,86500,5,False,True,False,False,False,True


The function *clean_trip_data* optionally fixes the trips that have duration mismatch (using the start and end timestamp of the trip) and also drops the rows that have issues considered as unrecoverable (negative or zero duration, unrealistic duration, missing timestamps, start after end date, dates out of range). Without further knowledge, it's diffucult to complete the information for these rows.

In [18]:
pogoh_df_cleaned = pc.clean_trip_data(pogoh_df_cleaned)

Fixed duration in 0 row(s).
Dropped 9 row(s) with unrecoverable issues.


In [ ]:
# Create a dataset where different problems exist
df_test_cleaning = pd.DataFrame({
    "start_date": [
        "2025-06-17 08:00:00",  # zero duration
        "2025-06-17 09:00:00",  # unrealistic duration
        None,                   # missing timestamp
        "2025-06-17 12:00:00",  # start after end
        "2010-06-17 08:00:00",  # datetime out of range
        "2025-06-17 15:00:00",  # duration mismatch (duration too small)
        "2025-06-17 16:00:00"   # clean row
    ],
    "end_date": [
        "2025-06-17 08:00:00",  # zero duration
        "2025-06-17 12:00:00",  # long duration (10800 sec = 3 hours)
        "2025-06-17 11:00:00",  # missing start
        "2025-06-17 11:00:00",  # end before start
        "2010-06-17 09:00:00",  # datetime out of range
        "2025-06-17 15:10:00",  # 600 sec duration (but says 100 sec)
        "2025-06-17 16:30:00"   # clean row
    ],
    "duration": [
        0,          # zero
        100000,     # unrealistic
        3600,       # OK duration but missing timestamp
        300,        # would be invalid (start after end)
        3600,       # out of range datetime
        100,        # mismatch (true is 600 sec)
        1800        # valid
    ]
})

pc.clean_trip_data(df_test_cleaning)

Fixed duration in 3 row(s).
Dropped 4 row(s) with unrecoverable issues.


,start_date,end_date,duration,original_index
0,2025-06-17 09:00:00,2025-06-17 12:00:00,10800,1
1,2025-06-17 15:00:00,2025-06-17 15:10:00,600,5
2,2025-06-17 16:00:00,2025-06-17 16:30:00,1800,6


Add *flag_trip_outliers* information

In [20]:
df_flag_test = pc.flag_trip_outliers(df_test_cleaning)
df_flag_test

,start_date,end_date,duration,is_short_duration_outlier,is_long_duration_outlier
0,2025-06-17 08:00:00,2025-06-17 08:00:00,0,True,False
1,2025-06-17 09:00:00,2025-06-17 12:00:00,100000,False,True
2,None,2025-06-17 11:00:00,3600,False,False
3,2025-06-17 12:00:00,2025-06-17 11:00:00,300,False,False
4,2010-06-17 08:00:00,2010-06-17 09:00:00,3600,False,False
5,2025-06-17 15:00:00,2025-06-17 15:10:00,100,False,False
6,2025-06-17 16:00:00,2025-06-17 16:30:00,1800,False,False


In [21]:
pc.summarize_trip_outliers(df_flag_test)

Trip Outlier Summary:
------------------------------
[FLAG] Short duration outliers: 1
[FLAG] Long duration outliers: 1
------------------------------


Add *add_time_features* information

In [22]:
pc.add_time_features(df_test_cleaning)

,start_date,end_date,duration,trip_duration_min,start_hour,end_hour,start_weekday,is_weekend
0,2025-06-17 08:00:00,2025-06-17 08:00:00,0,0.000000,8.0,8,1.0,False
1,2025-06-17 09:00:00,2025-06-17 12:00:00,100000,1666.666667,9.0,12,1.0,False
2,None,2025-06-17 11:00:00,3600,60.000000,NaN,11,NaN,False
3,2025-06-17 12:00:00,2025-06-17 11:00:00,300,5.000000,12.0,11,1.0,False
4,2010-06-17 08:00:00,2010-06-17 09:00:00,3600,60.000000,8.0,9,3.0,False
5,2025-06-17 15:00:00,2025-06-17 15:10:00,100,1.666667,15.0,15,1.0,False
6,2025-06-17 16:00:00,2025-06-17 16:30:00,1800,30.000000,16.0,16,1.0,False
